In [1]:
from ntscraper import Nitter
import pandas as pd

from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import re
import neattext.functions as nfx
import matplotlib.pyplot as plt

14-May-24 17:30:07 - NumExpr defaulting to 8 threads.


In [2]:
scraper = Nitter()

Testing instances:  92%|█████████▏| 71/77 [03:20<00:09,  1.57s/it]

14-May-24 17:33:43 - Certificate did not match expected hostname: nt.ggtyler.dev. Certificate: {'subject': ((('commonName', '4g.ggtyler.dev'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '03A4459CE6DA31CFF555DB78FF3F35BFEA88', 'notBefore': 'May 14 10:55:44 2024 GMT', 'notAfter': 'Aug 12 10:55:43 2024 GMT', 'subjectAltName': (('DNS', '4g.ggtyler.dev'),), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}


Testing instances:  95%|█████████▍| 73/77 [03:24<00:06,  1.74s/it]

14-May-24 17:33:44 - Certificate did not match expected hostname: nitter.uni-sonia.com. Certificate: {'subject': ((('commonName', '*.xserver.jp'),),), 'issuer': ((('countryName', 'JP'),), (('organizationName', 'CloudSecure Corporation'),), (('commonName', 'CloudSecure RSA Domain Validation Secure Server CA 2'),)), 'version': 3, 'serialNumber': 'ACA67AD2030638EE2DCE8E845B8299A6', 'notBefore': 'Mar 11 00:00:00 2024 GMT', 'notAfter': 'Apr 11 23:59:59 2025 GMT', 'subjectAltName': (('DNS', '*.xserver.jp'), ('DNS', 'xserver.jp')), 'OCSP': ('http://ocsp.sectigo.com',), 'caIssuers': ('http://crt.sectigo.com/CloudSecureRSADomainValidationSecureServerCA2.crt',)}


Testing instances:  99%|█████████▊| 76/77 [03:29<00:01,  1.81s/it]

14-May-24 17:33:49 - Certificate did not match expected hostname: nitter.tinfoil-hat.net. Certificate: {'subject': ((('commonName', 'jelly.tinfoil-hat.de'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '03F338CE809E122DC2875C50A27A840DD7A4', 'notBefore': 'Mar 15 22:40:16 2024 GMT', 'notAfter': 'Jun 13 22:40:15 2024 GMT', 'subjectAltName': (('DNS', 'jelly.tinfoil-hat.de'),), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}


Testing instances: 100%|██████████| 77/77 [03:30<00:00,  2.73s/it]


In [3]:
def get_tweets(name,modes,no):
    tweets = scraper.get_tweets(name,mode = modes, number = no)    
    final_tweets = []
    for tweet in tweets['tweets']:
        data = [tweet['link'],tweet['text'],tweet['date'], tweet['stats']['likes'],tweet['stats']['comments']]
        final_tweets.append(data)
    data = pd.DataFrame(final_tweets, columns = ['link','Tweets','Date','Likes','Count(Comments)'])
    return data

In [ ]:
name = input("Enter the username: ")
modes = input("enter the mode as 'term', 'hashtag' or 'user':")
no = int(input("Enter the number of tweets to analyze: "))

In [ ]:
# data = get_tweets('AdityaBirlaGrp','user',200) # Use 'term', 'hashtag', or 'user'.
# DrSJaishankar
# elon
data = get_tweets(name,modes,no) # Use 'term', 'hashtag', or 'user'.

In [ ]:
data.head()

In [ ]:
from translate import Translator
from langdetect import detect 

# Create a function to translate the tweets
def translate_text(text):
    try:
        detected_language = detect(text)
        translator = Translator(to_lang="en", from_lang=detected_language)
        translated_text = translator.translate(text)
        return translated_text
    except Exception as e:
        print("An error occurred during translation:", e)
        return text


data['Tweets_en'] = data['Tweets'].apply(translate_text)
data['Tweets_en']

In [ ]:
# Create a function to clean the tweets
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 return text


# Clean the tweets
data['Tweets_en'] = data['Tweets_en'].apply(cleanTxt)
data['Tweets_en'] = data['Tweets_en'].apply(nfx.remove_stopwords)
data['Tweets_en'] = data['Tweets_en'].apply(nfx.remove_punctuations)

# Show the cleaned tweets
data.sample(5)

In [ ]:
# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


# Create two new columns 'Subjectivity' & 'Polarity'
data['Subjectivity'] = data['Tweets_en'].apply(getSubjectivity)
data['Polarity'] = data['Tweets_en'].apply(getPolarity)

# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
data.sample(5)

In [ ]:
# # word cloud visualization

# # Remove extra whitespace from the allWords string
# allWords = ' '.join([twts for twts in data['Tweets']]).strip()
# mywordCloud = WordCloud( width=500, height=300, max_font_size=110).generate(allWords)
# plt.figure(figsize=(8,3))
# plt.imshow(mywordCloud, interpolation="bilinear")
# plt.axis('off')
# plt.show()

def generate_wordcloud(text):
    allWords = ' '.join([twts for twts in text]).strip()
    mywordCloud = WordCloud(width=500, height=300, max_font_size=110).generate(allWords)
    plt.figure(figsize=(8,3))
    plt.imshow(mywordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

generate_wordcloud(data['Tweets'])

In [ ]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def getAnalysis(score):
  if score < 2:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

def getEmoji(score):
  if score < 0:
    return '😔'
  elif score == 0:
    return '😐'
  else:
    return '🙂'
      
data['Analysis'] = data['Polarity'].apply(getEmoji)
# Show the dataframe
data.sample(5)

In [ ]:
import plotly.express as px
counts = data['Analysis'].value_counts()
fig = px.bar(x=counts.index, y=counts.values)
fig.update_layout(title='Sentiment Analysis',
                  xaxis_title='Sentiment',
                  yaxis_title='Counts')
fig.show()

In [ ]:
import plotly.graph_objects as go

def generate_bar_chart(data):
    counts = data['Analysis'].value_counts()
    values = counts.index

    fig = go.Figure(data=[go.Bar(x=values, y=counts.values, marker=dict(color=['blue', 'green', 'red', 'purple', 'yellow']), name=values[0])])
    fig.update_layout(title='Sentiment Analysis', xaxis_title='Sentiment', yaxis_title='Counts')
    fig.show()
    
generate_bar_chart(data)

In [ ]:
# import joblib

# def predict_with_model(df):
#     # Load the model
#     model = joblib.load('clf.pkl')
    
#     # Make predictions
#     predictions = model.predict(df)
    
#     # Define a mapping between numeric labels and sentiment categories
#     sentiment_mapping = {
#         0: 'Negative',
#         1: 'Neutral',
#         2: 'Positive'
#     }
    
#     # Convert numeric labels to sentiment categories
#     predicted_sentiments = [sentiment_mapping[label] for label in predictions]
    
#     return predicted_sentiments
def predict_with_model(text_list):
    # Load the model
    model = joblib.load('clf.pkl')
    # Make predictions
    predictions = model.predict(text_list)
    # Define a mapping between numeric labels and sentiment categories
    sentiment_mapping = {
        0: 'Negative',
        1: 'Neutral',
        2: 'Positive'
    }
    # Convert numeric labels to sentiment categories for each prediction
    predicted_sentiments = [sentiment_mapping[label] for label in predictions]
    return predicted_sentiments

data['predictions'] = predict_with_model(data['Tweets_en'])

In [ ]:
import plotly.express as px
counts = data['predictions'].value_counts()
fig = px.bar(x=counts.index, y=counts.values)
fig.update_layout(title='Sentiment Analysis',
                  xaxis_title='Sentiment',
                  yaxis_title='Counts')
fig.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_wordcloud(text):
    # Concatenate all words in the text data
    allWords = ' '.join([twts for twts in text]).strip()
    
    # Create a WordCloud object with appropriate font for multi-language support
    # You can specify a font file that supports multiple languages, such as Arial Unicode MS
    mywordCloud = WordCloud(width=500, height=300, max_font_size=110, font_path='arial.ttf').generate(allWords)
    
    # Display the word cloud
    plt.figure(figsize=(10, 6))
    plt.imshow(mywordCloud, interpolation='bilinear')
    plt.axis('off')  # Hide the axes
    plt.show()

# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# from langdetect import detect
# import re

# def generate_wordcloud(text):
#     # Concatenate all words in the text data
#     allWords = ' '.join(text)

#     # Filter out non-alphanumeric characters and extra spaces
#     allWords = re.sub(r'[^\w\s]', '', allWords)
#     allWords = re.sub(r'\s+', ' ', allWords)

#     # Detect the language of each word and create a WordCloud for each language
#     wordclouds = {}
#     for word in allWords.split():
#         try:
#             lang = detect(word)
#             if lang not in wordclouds:
#                 wordclouds[lang] = ''
#             wordclouds[lang] += word + ' '
#         except:
#             pass

#     # Generate WordCloud for each language
#     for lang, words in wordclouds.items():
#         mywordCloud = WordCloud(width=500, height=300, max_font_size=110, font_path='arial.ttf').generate(words)

#         # Display the word cloud
#         plt.figure(figsize=(10, 6))
#         plt.imshow(mywordCloud, interpolation='bilinear')
#         plt.title(f'Word Cloud for {lang}')
#         plt.axis('off')  # Hide the axes
#         plt.show()


# Example usage
text = ["Hello", "Bonjour", "Hola", "Ciao", "Привет", "प्रत्येक व्यक्ति को विचार और उसकी अभिव्यक्ति की"]
generate_wordcloud(text)


In [ ]:
final_df = data

In [ ]:
final_df['Tweets_en']=final_df['Tweets_en'].astype(str)
final_df['Tweets_en']=final_df['Tweets_en'].apply(lambda x:x.lower())

In [ ]:
import nltk
from nltk.stem.porter import *
ps=PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y) #returns string after stemming

In [ ]:
final_df['Tweets_en']= final_df['Tweets_en'].apply(stem)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_model = LabelEncoder()
final_df['Label'] = le_model.fit_transform(final_df['Analysis'])

In [ ]:
import pickle
# Load the pickle model
with open('clf.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
test = model.predict(final_df['Tweets_en'])
test

In [ ]:
import numpy as np

# Make predictions on the Tweets_en column
predictions = model.predict(final_df['Tweets_en'])

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions)

# Add the predicted labels to the final_df dataframe
final_df['Predicted_Label'] = classes[predicted_labels]

# Print the first 5 rows of the final_df dataframe
final_df

In [ ]:
classes = ['Negative', 'Neutral', 'Positive']

print(f"True Label: {final_df['Analysis'][0]}")
print(f'Predict Label: {classes[test[0]]}')

In [ ]:
data[

In [ ]:
data['Predicted_Analysis']=classes[test]

In [ ]:
data[['Tweets','Analysis']].sample(10)

In [ ]:
def Analyze_Tweets(name,modes,no):
    data = get_tweets(name,modes,no)
    
    data['Tweets_en'] = data['Tweets'].apply(translate_text)
    data['Tweets_en'] = data['Tweets_en'].apply(cleanTxt)
    data['Tweets_en'] = data['Tweets_en'].apply(nfx.remove_stopwords)
    data['Tweets_en'] = data['Tweets_en'].apply(nfx.remove_punctuations)
    
    data['Subjectivity'] = data['Tweets_en'].apply(getSubjectivity)
    data['Polarity'] = data['Tweets_en'].apply(getPolarity)
    data['Analysis'] = data['Polarity'].apply(getAnalysis)

    print(data)
    print(generate_wordcloud(data['Tweets']))
    print(generate_bar_chart(data))
    print(data[['Tweets','Analysis']].sample(10))

In [ ]:
Analyze_Tweets('opdaiIy','user',200)

In [ ]:
data

In [ ]:
def Analyze_Tweets1(data):
    data['Tweets_en'] = data['Tweets'].apply(translate_text)
    data['Tweets_en'] = data['Tweets_en'].apply(cleanTxt)
    data['Tweets_en'] = data['Tweets_en'].apply(nfx.remove_stopwords)
    data['Tweets_en'] = data['Tweets_en'].apply(nfx.remove_punctuations)
    
    data['Subjectivity'] = data['Tweets_en'].apply(getSubjectivity)
    data['Polarity'] = data['Tweets_en'].apply(getPolarity)
    data['Analysis'] = data['Polarity'].apply(getAnalysis)
    return data

In [ ]:
data = Analyze_Tweets1(data)

In [ ]:
data.columns